# APARTADO 2: WEB SCRAPING TRADICIONAL Y SCRAPING DE INTERFACES CON BEAUTIFULSOUP Y SELENIUM
## 2.1 Introducción
En caso de no disponer de una API que nos permita acceder a datos textuales en un sitio web, o si la información que nos ofrece la plataforma no se encuentra expuesta de forma estructurada, tenemos la opción de usar web scraping. Es una técnica que consiste en obtener información de forma directa de las páginas, gracias a la interpretación de la estructura y el contenido del documento HTML. Una vez descargado o renderizado el HTML, podemos apuntar a secciones o nodos específicos para extraer texto, enlaces u otros atributos.
En el contexto de la ingeniería de software, la habilidad de raspar sitios resulta valiosa para recopilar corpus de texto, datos de reseñas, foros de discusión, listados de productos, entre otros. El scraping puede presentarse en dos grandes escenarios:
- Sitios estáticos, en los que el HTML contiene la mayoría o la totalidad de la información deseada en el momento de la respuesta HTTP inicial.
- Sitios dinámicos, sustentados por JavaScript, lo que implica que parte del contenido se genere tras la carga de la página, o requiera la simulación de acciones del usuario (clics, scroll, inicios de sesión).
Para el primer caso, BeautifulSoup suele ser suficiente; para el segundo, suele necesitarse un navegador automatizado como Selenium. Existen situaciones híbridas donde, antes de decidir utilizar Selenium, conviene analizar las llamadas de red a través de BurpSuite o las Developer Tools del navegador, a fin de discernir si el contenido se podría obtener con requests directamente desde un endpoint interno.

## 2.2 Estructura de los sitios web (HTML, CSS, JavaScript)
### 2.2.1 Fundamentos de la estructura HTML
Un sitio web se describe con HTML (HyperText Markup Language). Este lenguaje define los nodos que componen la página, usualmente organizados de manera jerárquica (árbol DOM). Cada nodo se compone de:
1. Etiqueta de inicio (p. ej., `<div class="post">`), que incluye el nombre del elemento y atributos.
2. Contenido interno: puede ser texto crudo, otros nodos anidados (hijos) o ambos.
3. Etiqueta de cierre (p. ej., `</div>`), excepto en elementos vacíos (e.g. `<img />`).

La organización de HTML se clasifica a menudo en varios tipos de etiquetas, que cada desarrollador o framework utiliza para fines semánticos o de maquetación:

- Estructura y disposición general: páginas modernas suelen utilizar `<header>`, `<nav>`, `<main>`, `<section>`, `<article>`, `<aside>` y `<footer>` para señalar la intención de cada bloque. Por ejemplo, un `<header>` se encuentra al inicio de la página, un `<footer>` al final, `<nav>` para menús, `<article>` para contenido principal.
- Agrupación de contenido: etiquetas como `<div>` y `<span>` son elementos genéricos de bloque o en línea, respectivamente, para agrupar contenido. `<p>` indica párrafos. Muchas webs reusan `<div>` para todo tipo de “cajas” y `<span>` para “fragmentos en línea.”
- Semántica y resaltado: etiquetas como `<h1>`, `<h2>`, `<h3>` indican encabezados en distintos niveles. `<b>`, `<strong>` y `<i>` realzan palabras, aunque `<strong>` e `<em>` tienen un significado semántico distinto al meramente visual de `<b>` o `<i>`.
- Inserción de contenido externo: se hallan elementos como `<audio>`, `<video>`, `<iframe>` y `<img>`. De hecho, un `<iframe>` puede embeber otra página entera. Estos pueden presentar retos para scraping si el contenido real se aloja en un documento adicional o en un streaming.
- Tablas: se definen con `<table>`, `<tr>` (fila), `<td>` (celda). En sitios antiguos, las tablas se usaban para maquetar la página entera; en sitios modernos, se reservan para datos tabulares genuinos. Por ejemplo, una tabla de precios de productos o un ranking de usuarios.
- Formularios: `<form>` abarca todo el bloque de campos, `<input>` define un campo (texto, checkbox, etc.), `<select>` para menús desplegables, `<button>` o `<input type="submit">` para enviar. Para scraping, es clave entender cómo enviar datos al servidor (p. ej. un login) o cómo se estructuran los parámetros que se envían (e.g. `name="username"`).

https://itwebtutorials.mga.edu/html/chp2/document-structure.aspx


En el siguiente código de HTML se ilustra una estructura sencilla:

```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Web U-Tad</title>
</head>
<body>
  <header>
    <h1>Bienvenido a la U-tad!</h1>
    <nav>
      <ul>
        <li><a href="/index.html">Inicio</a></li>
        <li><a href="/about.html">Sobre nosotros</a></li>
      </ul>
    </nav>
  </header>
  <main>
    <section class="blog-entry">
      <h2>Primer post</h2>
      <p>Estudiar en la U-Tad es genial!</p>
      <span class="author">Creado por Jesús</span>
      <aside>Más información</aside>
    </section>
    <section class="blog-entry">
      <h2>Segundo post</h2>
      <p>Estoy mejorando mucho mi conocimiento en Python!</p>
      <span class="author">Creado por Jesús</span>
    </section>
  </main>
  <footer>
    <p>&copy; 2025 U-Tad.</p>
  </footer>
</body>
</html>
```

Este ejemplo refleja cómo se usan `<header>` y `<footer>` para la cabecera y pie de página. El `<nav>` aloja enlaces de navegación, `<main>` contiene el contenido principal, dividido en `<section>` con clase blog-entry y, dentro, `<h2>` para el título y `<p>` para el texto. Nodos como `<aside>` añaden información extra.

### 2.2.2 El rol de CSS y JavaScript
El CSS (Cascading Style Sheets) define la apariencia de los nodos (colores, márgenes, tipografía). Para el scraping, lo importante es poder localizar elementos a través de selectores. Por ejemplo, un bloque con clase blog-entry se selecciona con .blog-entry en CSS. Si deseamos un `<span>` de autor con clase author, en CSS se define .author. Al usar librerías como BeautifulSoup o Selenium, esos selectores simplifican la extracción: `soup.select(".blog-entry .author")`
CSS se caracteriza por ser un lenguaje simple, permitir el anidamiento de instrucciones, ser estándar en todos los navegadores y permitir personalizar la apariencia de las páginas. Además, se puede insertar dentro de una etiqueta HTML o en un fichero aparte.
La estructura de CSS pues se compone de selectores, reglas y valores como:
```css
/* Selector: p (selecciona todos los párrafos) */
p {
  /* Regla: color con el valor azul */
  color: blue;
}
```
JavaScript, se trata de un lenguaje de programación interpretado que nos permite interactuar con los elementos de la web y "darles vida". Gracias a esto, la web, internet y las redes sociales se han desarrollado hasta el nivel en el que se encuentran hoy en día.
Los sitios dinámicos usan JavaScript para alterar el DOM (Document Object Model, interfaz que facilita la representación, manteniendo una estructura definida, del documento) en tiempo de ejecución, insertar datos o responder a eventos. Este factor es crucial para el scraping: un `requests.get(url)` puede devolver un HTML vacío si la mayor parte del contenido se inyecta tras la carga con JavaScript. En tal caso, se requiere Selenium (o alguna técnica que reproduzca llamadas AJAX) para obtener el estado final del DOM.
JavaScript se caracteriza por ser un lenguaje de alto nivel, orientado a objetos y estar débilmente tipado. Además, es un lenguaje Interpretado que interactúa con el DOM y se ejecuta en cliente.

## 2.3 Herramientas de inspección (Dev Tools y BurpSuite)
Las Developer Tools (Chrome/Firefox) son el método más directo para explorar la estructura y el tráfico de una página. Nos ayudan a:
- Ver el DOM real en la pestaña “Elements” (o “Inspector” en Firefox).
- Seleccionar elementos y ver sus atributos, clases y estilos.
- Revisar peticiones HTTP en “Network,” confirmando si hay endpoints JSON, tokens, o secuencias de requests para la paginación.

Esta exploración inicial permite definir la estrategia de scraping: “¿está todo en un `<div class="post">`, o necesito descubrir requests AJAX con JSON?”

BurpSuite es una suite más avanzada orientada a pruebas de seguridad, pero muy útil para scraping. Configurando el navegador para pasar por el proxy de BurpSuite, podemos interceptar cada request y respuesta. Esto revela:

- Tokens en cabeceras o formularios.
- Endpoints internos que devuelven datos en JSON.
- Redirecciones a otros dominios.

Una vez hallamos esos endpoints internos, a menudo podemos replicar la petición con requests sin necesidad de Selenium ni parseo HTML. Por ejemplo, si la aplicación Angular llama a `GET /api/posts?page=2`, podríamos interceptar esa request en Burp, ver qué parámetros y cookies requiere, y replicar en Python. De esta forma, podemos ser capaces de autenticarnos o hacer otras acciones incluso aunque las APIs no estén pensadas o preparadas para ello mediante el uso o modificación de cookies, headers, etc.

## 2.4 Query selectors
Para scrapear de forma efectiva, necesitamos seleccionar los nodos de interés dentro del DOM de la página. Los query selectors son la vía principal para decirle a nuestras herramientas (un navegador, BeautifulSoup o Selenium) qué elementos queremos. Estos selectores se basan en la misma sintaxis que utiliza CSS, combinando etiquetas, clases, ids, atributos y pseudoelementos para localizar con precisión cualquier nodo del árbol HTML.
Un query selector es una cadena que describe uno o varios elementos del DOM conforme a reglas definidas en el estándar CSS. Los selectores pueden combinarse jerárquicamente (`div > p`) o unirse por lógica (`h2.title, p.description`). El navegador, BeautifulSoup y Selenium ofrecen métodos para usarlos:
- En el navegador, en la consola de DevTools, podemos emplear `document.querySelector("...")` (devuelve el primer elemento) o `document.querySelectorAll("...")` (devuelve todos los que coincidan).
- En BeautifulSoup, disponemos de `soup.select("...")` para obtener una lista de elementos, y `select_one("...")` para un único elemento.
- En Selenium, se puede emplear `driver.find_element("css selector", "...")` o `driver.find_elements("css selector", "...")`

La sintaxis de los selectores es muy flexible. Entre los selectores más comunes encontramos:

| Tipo de Selector | Símbolo / Ejemplo | Función | Ejemplo de uso |
|------------------|-------------------|---------|----------------|
| Por etiqueta (tag) | `"p", "div", "span"` | Selecciona todos los elementos de un tipo concreto de etiqueta. | CSS: `p { color: blue; }` <br> JS: `document.querySelectorAll("p")` <br> BeautifulSoup: `soup.select("p")` |
| Por id | `#header, #main, #footer` | Selecciona un elemento cuyo atributo id coincida con el especificado. (El id debería ser único en la página). | CSS: `#header { background: black; }` <br> JS: `document.querySelector("#header")` <br> BeautifulSoup: `soup.select("#header")` |
| Por clase | `.blog-entry, .author` | Selecciona todos los elementos que posean la clase dada. Una etiqueta puede tener varias clases: `<div class="blog-entry post">`. | CSS: `.blog-entry { margin-bottom: 20px; }` <br> JS: `document.querySelectorAll(".blog-entry")` <br> BeautifulSoup: `soup.select(".blog-entry")` |
| Combinado (clases) | `.card.highlight` | Selecciona elementos con ambas clases. `<div class="card highlight">`. | CSS: `.card.highlight { border: 1px solid; }` <br> JS: `document.querySelectorAll(".card.highlight")` <br> BS: `soup.select(".card.highlight")` |
| Jerárquico (descend.) | `"div p"` | Selecciona `<p>` que estén dentro de un `<div>` en cualquier nivel de anidación. | CSS: `div p { color: red; }` <br> JS: `document.querySelectorAll("div p")` <br> BS: `soup.select("div p")` |
| Jerárquico (hijo) | `"div > p"` | Selecciona `<p>` que sean hijos directos de `<div>`. | CSS: `div > p { font-size: 14px; }` <br> JS: `document.querySelectorAll("div > p")` <br> BS: `soup.select("div > p")` |
| Hermano adyacente | `"div + p"` | Selecciona `<p>` que aparezca justo después de un `<div>` (en el mismo nivel). | CSS: `div + p { margin-top: 5px; }` <br> JS: `document.querySelectorAll("div + p")` <br> BS: `soup.select("div + p")` |
| Hermano general | `"div ~ p"` | Selecciona todos los `<p>` posteriores al `<div>` dentro del mismo padre. | CSS: `div ~ p { color: green; }` <br> JS: `document.querySelectorAll("div ~ p")` <br> BS: `soup.select("div ~ p")` |
| Selector de atributo | `img[src], a[href="..."]` | Selecciona elementos que posean un atributo en particular. Se puede especificar el valor, p. ej. `a[href="login.html"]`. | CSS: `img[src] { border: 1px solid #ccc; }` <br> JS: `document.querySelectorAll('img[src]')` <br> BS: `soup.select('img[src]')` <br> Combinado: `soup.select('a[href="login.html"]')` |
| Múltiple | `"h2, p.desc, div.content"` | Selecciona `h2` o `p.desc` o `div.content`, es decir, la unión de tres criterios distintos. | CSS: `h2, p.desc, div.content { margin:10px; }` <br> JS: `document.querySelectorAll("h2, p.desc, div.content")` <br> BS: `soup.select("h2, p.desc, div.content")` |

Supongamos que queremos `<a>` con clase "button" y cuyo atributo `href` contenga la palabra "product" pero que, además, sea un hijo directo de un `<li>` que esté dentro de un `<ul>` de clase "nav-list". El selector podría ser:

`ul.nav-list > li > a.button[href*="product"]`

Una manera sencilla de validar los query selectors es desde la consola de desarrolladores del navegador. Para ello deberemos:

1. Para este ejemplo iremos a la web: https://parascrapear.com/
2. Abrir las herramientas de desarrollador y buscar la consola con “F12”, “Ctrl+Shift+I”, dependerá de nuestro navegador.
3. En la consola debemos escribir `document.querySelectorAll("<TU_QUERY_SELECTOR>")`. Antes de ejecutar el comando ya obtenemos una pequeña vista previa de cuanto vamos a obtener



4. Obtengamos diferentes secciones de la página:

```javascript
// Citas
document.querySelectorAll("blockquote p > q")
```


```javascript
// Categorias
document.querySelectorAll("blockquote > p a.cat")
// Autores
document.querySelectorAll("blockquote > footer a.author")
// Siguiente página
document.querySelectorAll("a.next")
```

De esta forma, podemos acceder a los nodos para posteriormente extraer sus datos. Si quisiésemos acceder al texto, deberíamos leer el elemento y la propiedad correspondiente. Por ejemplo:
```javascript
// Texto de siguiente página
document.querySelectorAll("a.next")[0].text
// Output
'Siguiente →'
```

## 2.5 Scraping tradicional (HTML estático) con BeautifulSoup
BeautifulSoup es una librería Python para parsear HTML y XML de forma sencilla. Se encarga de convertir el HTML en un objeto DOM que podemos recorrer con métodos de búsqueda y selectores. Dado que en muchos sitios el contenido estático se encuentra en un HTML final, no necesitamos ejecutar JavaScript ni un navegador completo.
El proceso típico sería:
1. Descargamos la página con `requests.get(url).text`.
2. Creamos un `BeautifulSoup(html, \"html.parser\")`.
3. Buscamos nodos con `.find()`, `.find_all()`, `.select()`, extrayendo texto o atributos.

Esta técnica es simple, rápida y eficiente, siempre que la página no requiera JS o interacción dinámica para mostrar el contenido. Gracias a los Query Selectors podremos identificar los nodos deseados y extraer su información.

En cuanto al uso, podemos extraer información de atributos mediante `element[\"href\"]` o `element.get(\"href\")`. También podemos tratar el texto gracias a `.get_text(strip=True)` quita espacios extra. Para navegar, tenemos las opciones de `.parent`, `.children`, `.next_sibling`, y `.previous_sibling` se recorre el árbol, si la estructura es compleja.

Para instalar la librería, simplemente deberemos ejecutar:


In [ ]:
!pip install beautifulsoup4

Si utilizamos el mismo ejemplo hecho con los Query Selectors, podemos obtener todo el contenido de los nodos identificados de la siguiente manera:


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.parascrapear.com"
res = requests.get(url)
html = BeautifulSoup(res.text, 'html.parser')

citas = html.select("blockquote p > q")
citas = [x.text for x in citas]
print(f"Citas: {citas}")

categorias = html.select("blockquote > p a.cat")
categorias = [x.text for x in categorias]
print(f"Categorias: {categorias}")

autores = html.select("blockquote > footer a.author")
autores = [x.text for x in autores]
print(f"Autores: {autores}")

next_page_data = html.select("a.next")
next_page = f'{url}{[x["href"] for x in next_page_data][0]}'
print(f"Pagina siguiente: {next_page}")

Como podemos ver en el caso anterior, una vez extraídos los datos, observamos que aún sigue habiendo una “Página siguiente” que probablemente tenga más información. Una manera muy común para scrapear, sería hacer un pipeline que vaya extrayendo información hasta que no haya más páginas. Lo podemos ver en el siguiente ejemplo:


In [ ]:
# import requests
# from bs4 import BeautifulSoup
import pandas as pd

def scrape_page(url):
    res = requests.get(url)
    html = BeautifulSoup(res.text, 'html.parser')

    blockquotes = html.select("blockquote")
    quotes = []
    cat = []
    authors = []
    for bq in blockquotes:
        citas = bq.select_one("p > q")
        quotes.append(citas.text)

        categorias = bq.select_one("p a.cat")
        cat.append(categorias.text)

        autores = bq.select_one("footer a.author")
        authors.append(autores.text)

    next_node = html.select("a.next")

    if len(next_node) > 0:
        next_page = f'{url}{next_node[0]["href"]}'
    else:
        next_page = None

    return quotes, cat, authors, next_page

url = "https://www.parascrapear.com/"

total_quotes = []
total_cats = []
total_authors = []
while url is not None:
    quotes, cat, authors, url = scrape_page(url)
    total_authors += authors
    total_quotes += quotes
    total_cats += cat

df = pd.DataFrame({
    "quotes": total_quotes,
    "cats": total_cats,
    "authors": total_authors
})

df

## 2.6 Scraping dinámico con Selenium
Selenium es un marco de automatización de navegadores que permite simular la interacción de un usuario real. Esto resulta necesario cuando las páginas dependen de JavaScript, realizan peticiones AJAX después de la carga inicial, o exigen iniciar sesión para mostrar contenido. Selenium abre un navegador (Chrome, Firefox, Edge) o su versión headless, y ejecuta el mismo código JS que el usuario vería en pantalla.

La razón de usar Selenium en lugar de requests + BeautifulSoup es que:

- Algunos sitios no cargan datos en el HTML inicial, sino que se los traen con JavaScript.
- Hay sitios que solicitan logins, captchas o clicks en botones de “Load More.”
- Se requieren acciones complejas, como seleccionar opciones en menús y esperar un response asíncrono.

Selenium provee métodos para encontrar elementos (driver.find_element(...)) y leer propiedades como .text. Tras la ejecución de scripts, la página deviene en un estado final que podemos examinar.

Además, Selenium dispone de dos modos principales:

- Headless=True, con este modo toda la interacción con el navegador se hace “por detrás” de tal modo que en pantalla no se aprecia nada.
- Headless=False, con este modo se abre una pestaña del navegador y se puede ver como Selenium interactúa con los componentes de la página web.

Para instalar selenium simplemente deberemos ejecutar:


In [ ]:
# Preparar Google Colab para arrancar el código de Selenium
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Además, fuera de colab, será necesario tener instalado el ChromeDriver siguiendo: https://developer.chrome.com/docs/chromedriver/get-started

La configuración e inicialización de Selenium se puede resumir en:


In [3]:
from selenium import webdriver
# Inicializamos el ChromeDriver (o el driver que necesitemos según el navegador)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #No arrancamos el browser
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
wd = webdriver.Chrome(options=chrome_options) #'chromedriver',
# Obtenemos la web deseada
wd.get("https://www.google.com")
# Cerramos el driver para liberar los recursos utilizados
wd.quit()

En cuanto a la navegación y localización de elementos, Selenium ofrece numerosas opciones. De entre ellas, las básicas son:

- driver.get(url): abre la página especificada.
- driver.back(): navega a la página anterior en el historial.
- driver.forward(): avanza a la siguiente página en el historial.
- driver.refresh(): recarga la página actual.
- Para la localización de elementos:
  - find_element(By.ID, "id")
  - find_element(By.NAME, "name")
  - find_element(By.XPATH, "xpath")
  - find_element(By.LINK_TEXT, "link text")
  - find_element(By.PARTIAL_LINK_TEXT, "partial link text")
  - find_element(By.TAG_NAME, "tag name")
  - find_element(By.CLASS_NAME, "class name")
  - find_element(By.CSS_SELECTOR, "css selector"), este sería el equivalente a los Query Selectors
  
¿Cómo se resolvería el caso de scrapeme.live con selenium? Veamos el siguiente ejemplo:


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd

# Configuramos el WebDriver
def create_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') #No arrancamos el browser
    chrome_options.add_argument('--no-sandbox')
    chrome_options.headless = True
    wd = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome(options=chrome_options)
    return wd

url = "https://scrapeme.live/shop/Bulbasaur/"

driver = create_driver()
driver.get(url)


In [ ]:
driver.title # If it gives you "Privacy Error", then run the previous cell again.

In [ ]:
pokemon_name = driver.find_element(by=By.CSS_SELECTOR, value=".summary .product_title").text
print(f"Pokemon Name: {pokemon_name}")

pokemon_price = driver.find_element(by=By.CSS_SELECTOR, value="p.price").text
print(f"Pokemon Price: {pokemon_price}")

# Debemos cerrar el driver, de lo contrario no liberaremos los recursos del PC
driver.quit()

Del mismo modo, si queremos definir un pipeline en el que scrapear todas las páginas para obtener la información de todos los elementos:


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd

# Configuramos el WebDriver
def create_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') #No arrancamos el browser
    chrome_options.add_argument('--no-sandbox')
    chrome_options.headless = True
    wd = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome(options=chrome_options)
    return wd

# Función para obtener la información de un Pokémon
def get_pokemon_info(url):
    driver = create_driver()
    driver.get(url)

    try:
        pokemon_name_qs = ".summary .product_title"
        pokemon_price_qs = "p.price"
        pokemon_desc_qs = ".summary .woocommerce-product-details__short-description p"
        pokemon_stock_qs = ".summary .stock"
        pokemon_image_qs = ".woocommerce-product-gallery__image .wp-post-image"
        pokemon_price_curr_qs = ".woocommerce-Price-currencySymbol"

        pokemon_name = driver.find_element(By.CSS_SELECTOR, pokemon_name_qs).text
        print(f"Scraping {pokemon_name}")

        pokemon_price_curr = driver.find_element(By.CSS_SELECTOR, pokemon_price_curr_qs).text
        pokemon_price = driver.find_element(By.CSS_SELECTOR, pokemon_price_qs).text
        pokemon_price = float(pokemon_price.replace(pokemon_price_curr, "").strip())

        pokemon_desc = driver.find_element(By.CSS_SELECTOR, pokemon_desc_qs).text
        pokemon_stock = int(driver.find_element(By.CSS_SELECTOR, pokemon_stock_qs).text.split(" ")[0])
        pokemon_image = driver.find_element(By.CSS_SELECTOR, pokemon_image_qs).get_attribute("src")

    except Exception as e:
        driver.quit()
        return f"Error with page: {url}, {e}"

    driver.quit()
    return {"name": pokemon_name,
            "image_url": pokemon_image,
            "description": pokemon_desc,
            "price": pokemon_price,
            "currency": pokemon_price_curr,
            "stock": pokemon_stock}

# Función para obtener los elementos de la página
def get_page_elements(url):
    driver = create_driver()
    correct= False
    #while()
    driver.get(url)
    print(driver.title)

    try:
        print(f"Scraping: {url}")

        urls_qs = "ul .product .woocommerce-LoopProduct-link"
        next_url_qs = ".page-numbers .next"

        urls = [elem.get_attribute("href") for elem in driver.find_elements(By.CSS_SELECTOR, urls_qs)]

        next_url_elem = driver.find_elements(By.CSS_SELECTOR, next_url_qs)
        next_url = next_url_elem[0].get_attribute("href") if next_url_elem else None

    except Exception as e:
        driver.quit()
        return f"Error with: {url}, {e}"

    driver.quit()
    return urls, next_url

# Scrapeamos todas las páginas
next_page = "https://scrapeme.live/shop/"
pokemons_list = []


while next_page is not None:
    try:
      urls, next_page = get_page_elements(next_page)
      for url in urls:
          pokemon_info = get_pokemon_info(url)
          pokemons_list.append(pokemon_info)
      break # Solo leemos la primera página
    except Exception as e:
      #print("Error")
      print(f"Error scraping page: {e}")
      print("----------")

df = pd.DataFrame(pokemons_list)
df

La principal diferencia que podemos observar es a nivel de rendimiento ya que Selenium va a tardar muchos minutos más que BeatifulSoup. Para el caso de los Pokemon en Scrapeme.live estamos hablando de que **puede tardar del orden de horas** en recorrer las casi 50 páginas.


## 2.7 Selenium VS BeautifulSoup
Elegir la herramienta adecuada es fundamental para garantizar la eficiencia y efectividad del proceso de extracción de datos. BeautifulSoup y Selenium son dos de las bibliotecas más populares utilizadas para el parsing de contenido web, cada una con sus propias ventajas y limitaciones. Mientras que BeautifulSoup se destaca por su simplicidad y velocidad en el manejo de páginas estáticas, Selenium ofrece una capacidad superior para interactuar con sitios dinámicos que dependen de JavaScript y requieren acciones de usuario más complejas. Para facilitar la toma de decisiones informadas, analizaremos la siguiente tabla:

| Aspecto                | BeautifulSoup                                      | Selenium                                      |
|------------------------|----------------------------------------------------|-----------------------------------------------|
| Necesidad de JS        | Para webs estáticas sin necesidad de JS            | Para webs dinámicas con necesidad de JS       |
| Velocidad              | Muy rápida                                         | Lenta                                         |
| Interactividad         | No puede interactuar                               | Sí puede interactuar con botones y otros      |
| Necesidad de recursos  | Muy baja                                           | Muy alta                                      |
| Casos de uso           | Sencillos, para páginas con HTML                   | Complejos para webs modernas                  |


## 2.8 Extra
En la práctica, hay escenarios complejos donde la página expone parte de la información en JavaScript, y además requiere un token de sesión que se obtiene al iniciar la página. Para analizar el funcionamiento y saber qué solución debemos implementar, podremos realizar el siguiente proceso:
1. Configurar el navegador para redirigir el tráfico por el proxy de BurpSuite.
2. Navegar manualmente por la web e interceptar las requests. Si se detecta un GET /api/data?page=2 que devuelve un JSON con el contenido en lugar de HTML, podemos replicarlo en un script con requests.
3. Extraer el token “X-CSRF-Token” o “Authorization Bearer” desde el interceptado.
4. Probar con un script Python que incluya las cabeceras y parámetros descubiertos.
5. Descartar Selenium si no es preciso, o mantenerlo si la web requiere iniciar sesión mediante un formulario JS, en cuyo caso Selenium reproduciría la acción de login, y el proxy de BurpSuite confirmaría la existencia de cookies o tokens.
De esta manera, BurpSuite funciona como herramienta forense para descubrir qué pasa en el “backstage” de la aplicación web. Después, se decide si bastan requests y BeautifulSoup para parsear la respuesta HTML/JSON, o si se precisa Selenium para secuencias más intrincadas.

## 2.9 Ética y legalidad
La extracción masiva de datos puede topar con límites técnicos y legales. Ciertos sitios prohíben explicitamente el scraping o su reutilización en sus términos de servicio. Además, existe el archivo robots.txt, un mecanismo por el que se sugiere qué rutas el scraper no debe acceder. Aunque no es legalmente vinculante en todos los casos, se considera buena práctica respetarlo.
Saturar un sitio con demasiadas requests en poco tiempo puede considerarse un ataque (como un DDoS) o perjudicar su rendimiento. Se recomienda introducir pausas y no descargar miles de páginas simultáneamente. Por último, se recogen datos que identifiquen personas, se deben conocer las regulaciones (como GDPR en Europa) para no incurrir en violaciones de privacidad.

## 2.10 Enlaces de interés
• BurpSuite community edition: https://portswigger.net/burp/communitydownload

• BeautifulSoup documentation: https://beautiful-soup-4.readthedocs.io/

• Selenium documentation: https://www.selenium.dev/documentation/

• Ética en scraping: https://www.meritdata-tech.com/resources/blog/data/web-scraping-best-practices-ethical-data-collection/
